In [ ]:
#%pip install -q langchain-google-genai

In [1]:
from dotenv import load_dotenv
import os
# .env 파일을 불러와서 환경 변수로 설정
load_dotenv(dotenv_path='../.env')

GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
print(GOOGLE_API_KEY[:4])

AIza


In [4]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
    
# API 키 설정
# os.environ["GOOGLE_API_KEY"] = "your-google-api-key"

# 모델 초기화
llm = ChatGoogleGenerativeAI(
    #model="gemini-1.5-flash",  # 또는 "gemini-pro-vision"
    model="gemini-2.5-pro",
    temperature=0.3    
)

# 프롬프트 설정
prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 AI 전문가입니다."),
    ("human", "{topic}은(는) 무엇인가요?")
])

# 체인 실행
chain = prompt | llm
response = chain.invoke({"topic": "LangChain과 LangGraph"})

print(" Google Gemini Response:")
print(response.content)

Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits.
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 2
Please retry in 36.261444558s. [violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerMinutePerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-2.5-pro"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_value: 2
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 36
}
].
Retrying langchain_google_genai.chat_models._chat_with_retry.<locals

 Google Gemini Response:
네, AI 전문가로서 LangChain과 LangGraph에 대해 명확하고 깊이 있게 설명해 드리겠습니다.

이 둘은 별개의 것이 아니라, **LangGraph가 LangChain을 기반으로 더 복잡한 작업을 수행하기 위해 만들어진 확장 라이브러리**라는 관계를 먼저 이해하는 것이 중요합니다.

---

### 1. LangChain (랭체인): LLM 애플리케이션을 위한 '레고 블록'

**LangChain은 한마디로 '대규모 언어 모델(LLM)을 활용한 애플리케이션을 더 쉽고 강력하게 만들 수 있도록 도와주는 개발 프레임워크'입니다.**

LLM 자체는 단순히 텍스트를 입력받아 텍스트를 생성하는 기능만 가지고 있습니다. 하지만 우리가 원하는 실제 애플리케이션(챗봇, 문서 요약, 코드 생성 등)을 만들려면 LLM 외에 여러 가지 구성 요소가 필요합니다. LangChain은 이 구성 요소들을 마치 **레고 블록처럼 조립**하여 복잡한 애플리케이션을 만들 수 있게 해줍니다.

#### 핵심 개념 및 주요 구성 요소 (레고 블록들)

*   **Models**: OpenAI의 GPT, Google의 Gemini 등 다양한 LLM 모델에 쉽게 연결할 수 있는 인터페이스입니다.
*   **Prompts**: LLM에 전달할 지시문(프롬프트)을 효과적으로 관리하고, 변수를 넣어 동적으로 생성할 수 있는 템플릿 기능입니다.
*   **Chains**: LangChain의 핵심 개념으로, **'LLM 호출'과 다른 구성 요소들을 순차적으로 연결한 작업 흐름**을 의미합니다. 예를 들어, `(1)사용자 질문 받기 -> (2)프롬프트 템플릿에 질문 삽입 -> (3)LLM에 질문 전달 -> (4)LLM 답변 받기 -> (5)답변 형식 파싱` 과 같은 일련의 과정을 하나의 '체인'으로 묶을 수 있습니다.
*   **Indexes & Retrievers**: LLM이 학습하지 않은 외부 데이터(PDF, DB, 웹사이트 등)를 참조하여 답

#### Gemini 모델별 특징

* gemini-1.5-flash: 빠른 응답, 일반적인 작업에 적합
* gemini-1.5-pro: 더 정확하고 복잡한 추론 작업
* gemini-pro-vision: 이미지 처리 및 멀티모달 작업

In [ ]:
import os
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser

# API 키 설정
# os.environ["GOOGLE_API_KEY"] = "your-google-api-key"

# 기본 모델 설정
llm = ChatGoogleGenerativeAI(
    #model="gemini-1.5-flash",
    model="gemini-2.5-pro",
    temperature=0.7
)

print("=" * 50)
print("예제 1: 기본 대화형 챗봇")
print("=" * 50)

# 대화형 프롬프트
chat_prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 친근하고 도움이 되는 AI 어시스턴트입니다."),
    ("human", "{user_input}")
])

chat_chain = chat_prompt | llm | StrOutputParser()
response1 = chat_chain.invoke({"user_input": "파이썬으로 리스트를 정렬하는 방법은?"})
print("응답:", response1)

print("\n" + "=" * 50)
print("예제 2: JSON 구조화 출력")
print("=" * 50)

json_prompt = PromptTemplate(
    template="""
다음 정보를 JSON 형태로 변환하세요:
{company_info}

형식: {{"name": "회사명", "year": "연도", "location": "위치"}}
""",
    input_variables=["company_info"]
)

json_chain = json_prompt | llm | StrOutputParser()
company_text = "네이버는 1999년에 설립된 한국의 IT 기업이며 본사는 경기도 성남에 있습니다."
response2 = json_chain.invoke({"company_info": company_text})
print("JSON 결과:", response2)



In [ ]:
print("\n" + "=" * 50)
print("예제 3: 번역 체인")
print("=" * 50)

translate_prompt = ChatPromptTemplate.from_template(
    "다음 텍스트를 {target_language}로 번역하세요: {text}"
)

translate_chain = translate_prompt | llm | StrOutputParser()
original = "Hello, how are you today?"
translated = translate_chain.invoke({
    "text": original, 
    "target_language": "한국어"
})
print("번역 결과:", translated)

print("\n" + "=" * 50)
print("예제 4: 감정 분석")
print("=" * 50)

emotion_prompt = ChatPromptTemplate.from_template("""
텍스트: {text}
감정을 분석하고 [긍정/부정/중립]과 1-10점수를 매기세요.
""")

emotion_chain = emotion_prompt | llm | StrOutputParser()
test_text = "오늘 프로젝트가 성공적으로 완료되어서 정말 기쁩니다!"
emotion_result = emotion_chain.invoke({"text": test_text})
print("감정 분석:", emotion_result)

print("\n" + "=" * 50)
print("예제 5: 코드 생성")
print("=" * 50)

code_prompt = ChatPromptTemplate.from_template("""
{language}로 {task} 기능을 구현하는 간단한 코드를 작성하세요.
""")

code_chain = code_prompt | llm | StrOutputParser()
code_result = code_chain.invoke({
    "language": "Python",
    "task": "두 숫자의 최대공약수를 구하는"
})
print("생성된 코드:")
print(code_result)

print("\n" + "=" * 50)
print("예제 6: 창의적 콘텐츠 생성")
print("=" * 50)


In [ ]:

# 창의적 생성용 높은 temperature
llm_creative = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    temperature=0.9
)

creative_prompt = ChatPromptTemplate.from_template(
    "{topic}에 대한 창의적인 {content_type}를 {style} 스타일로 작성하세요."
)

creative_chain = creative_prompt | llm_creative | StrOutputParser()
creative_result = creative_chain.invoke({
    "topic": "미래의 교통수단",
    "content_type": "아이디어",
    "style": "혁신적이고 실현 가능한"
})
print("창의적 아이디어:", creative_result)

print("\n" + "=" * 50)
print("Gemini 모델 옵션")
print("=" * 50)
print("• gemini-1.5-flash: 빠른 응답, 일반 작업")
print("• gemini-1.5-pro: 정확한 분석, 복잡한 추론")
print("• gemini-pro-vision: 이미지 처리 가능")
print("• temperature: 0.1(정확) ~ 0.9(창의적)")
print("=" * 50)